In [46]:
from sqlalchemy import create_engine
from sqlalchemy import MetaData
from time import sleep
from geopy.geocoders import Nominatim
import pandas as pd
import numpy as np
import geopandas
import folium
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import streamlit as st

In [73]:
conexion = create_engine("postgresql://{user}:{pw}@{host}:{port}/{db}"
                         .format(user="postgres",
                                 pw="postgres",
                                 host="database-grupo5.cgmzd7suyc4v.us-east-1.rds.amazonaws.com",
                                 port="5432",
                                 db="postgres",
                                 echo=False))
if conexion.connect():
  print("conexion exitosa")

conexion exitosa


In [74]:
g_n=pd.read_csv('https://raw.githubusercontent.com/MAKACT/proyectoConsumoEnergiaEquipo5/main/Consumo%20Gas%20Natural%20Exajoules.csv')
g_n=g_n.drop(columns={'2021.1','2011-21','2021.2'})
g_n.fillna(0, inplace=True)
g_n.replace("-", 0, inplace=True)
g_n.replace("^", 0, inplace=True)
g_n.replace("♦", 0, inplace=True)
indexN=g_n[g_n['Pais']=='Other Europe (BP)'].index
g_n.drop(indexN, inplace=True)
indexN=g_n[g_n['Pais']=='Other CIS (BP)'].index
g_n.drop(indexN, inplace=True)
indexN=g_n[g_n['Pais']=='Other Middle East (BP)'].index
g_n.drop(indexN, inplace=True)
indexN=g_n[g_n['Pais']=='Other Western Africa (BP)'].index
g_n.drop(indexN, inplace=True)
indexN=g_n[g_n['Pais']=='Other Northern Africa (BP)'].index
g_n.drop(indexN, inplace=True)
indexN=g_n[g_n['Pais']=='Other Asia Pacific (BP)'].index
g_n.drop(indexN, inplace=True)

g_n

,Pais,1965,1966,1967,1968,1969,1970,1971,1972,1973,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Canada,0.78,0.85,0.91,1.01,1.13,1.25,1.33,1.50,1.59,...,3.58,3.80,3.95,3.97,3.78,3.96,4.16,4.22,4.08,4.29
1,Mexico,0.29,0.31,0.26,0.28,0.35,0.36,0.37,0.40,0.44,...,2.65,2.80,2.84,2.91,2.99,3.10,3.15,3.17,3.01,3.18
2,US,14.97,16.14,17.04,18.24,19.63,20.70,21.34,21.55,21.38,...,24.77,25.45,26.00,26.77,26.97,26.64,29.58,30.62,29.95,29.76
3,Argentina,0.15,0.16,0.17,0.19,0.19,0.21,0.23,0.26,0.29,...,1.64,1.66,1.66,1.68,1.74,1.74,1.75,1.68,1.58,1.65
4,Brazil,0,0,0,0,0,0,0,0.01,0.01,...,1.17,1.38,1.46,1.55,1.34,1.35,1.29,1.29,1.13,1.46
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87,Sri Lanka,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88,Taiwan,0.01,0.01,0.01,0.02,0.02,0.02,0.04,0.05,0.05,...,0.64,0.65,0.68,0.73,0.75,0.84,0.85,0.84,0.90,0.98
89,Thailand,0,0,0,0,0,0,0,0,0,...,1.75,1.76,1.80,1.83,1.82,1.80,1.80,1.83,1.69,1.69
90,Vietnam,0,0,0,0,0,0,0,0,0,...,0.32,0.34,0.35,0.37,0.37,0.34,0.35,0.35,0.32,0.26


In [75]:
año=pd.read_sql_query('SELECT *FROM "Años";', con=conexion)
año

,Id_anio,Anio
0,1,1965
1,2,1966
2,3,1967
3,4,1968
4,5,1969
5,6,1970
6,7,1971
7,8,1972
8,9,1973
9,10,1974


In [76]:
años.dtypes

Id_anio    int64
Anio       int64
dtype: object

In [77]:
def cambiar_nombre_x_id_años(df_años,df):
    for i,e in enumerate(df_años['Anio']):
        id= df_años.loc[i,'Id_anio']
        df.loc[df['Año']==e,'Id_anio']=id
    return df

In [78]:
años=list(np.arange(1965,1981))
años=list(map(str,años))
g_n.drop(años, axis=1, inplace=True)
años=list(np.arange(1981,2022))
años=list(map(str,años))
g_n = pd.melt(g_n, id_vars='Pais', value_vars=años, var_name="Año", value_name= "Consumo_Gas")
g_n

,Pais,Año,Consumo_Gas
0,Canada,1981,1.74
1,Mexico,1981,0.87
2,US,1981,18.75
3,Argentina,1981,0.42
4,Brazil,1981,0.03
...,...,...,...
3767,Sri Lanka,2021,0
3768,Taiwan,2021,0.98
3769,Thailand,2021,1.69
3770,Vietnam,2021,0.26


In [79]:
g_n.dtypes

Pais           object
Año            object
Consumo_Gas    object
dtype: object

In [80]:
g_n['Año']=g_n['Año'].astype(int)
g_n.dtypes

Pais           object
Año             int32
Consumo_Gas    object
dtype: object

In [81]:
gn=cambiar_nombre_x_id_años(año,g_n)
gn

,Pais,Año,Consumo_Gas,Id_anio
0,Canada,1981,1.74,17.0
1,Mexico,1981,0.87,17.0
2,US,1981,18.75,17.0
3,Argentina,1981,0.42,17.0
4,Brazil,1981,0.03,17.0
...,...,...,...,...
3767,Sri Lanka,2021,0,57.0
3768,Taiwan,2021,0.98,57.0
3769,Thailand,2021,1.69,57.0
3770,Vietnam,2021,0.26,57.0


In [82]:
carbon=pd.read_sql_query('SELECT *FROM "Energia_Carbon";', con=conexion)
carbon.rename(columns={'Id_Anio':'Id_anio'},inplace=True)
carbon

,Id_Energia_Carbon,Id_Pais,Id_anio,Consumo_Carbon,Produccion_Carbono
0,1,44,17,0.95,0.97
1,2,160,17,0.09,0.07
2,3,297,17,15.94,18.42
3,4,35,17,0.24,0.12
4,5,54,17,0.10,0.12
...,...,...,...,...,...
1430,1431,225,57,0.67,0.14
1431,1432,262,57,3.04,0.02
1432,1433,276,57,0.81,0.15
1433,1434,304,57,2.15,1.12


In [83]:
petroleo=pd.read_sql_query('SELECT *FROM "Energia_petroleo";', con=conexion)
petroleo

,ID_Energia_petroleo,Id_Pais,Id_anio,Consumo_Petroleo,Produccion_Petroleo,Reservas_Petroleo
0,1,44,1,2.474186,2.054378,0.0
1,2,160,1,0.705634,0.808353,0.0
2,3,297,1,25.706527,20.128443,0.0
3,4,11,1,1.018257,0.616314,0.0
4,5,35,1,0.683304,0.214370,0.0
...,...,...,...,...,...,...
2275,2276,124,57,3.284773,1.545250,0.0
2276,2277,0,57,1.703794,1.279520,0.0
2277,2278,276,57,2.710886,0.888742,0.0
2278,2279,304,57,1.036121,0.428740,0.0


In [84]:
gas=pd.read_sql_query('SELECT *FROM "Gas_Natural";', con=conexion)
gas

,ID_Gas_Natural,Id_Pais,Id_anio,Reserva_Gas,Produccion_gas,Consumo_gas
0,1,44,16,2.4,2.4,1.79
1,2,160,16,1.8,1.8,0.80
2,3,297,16,5.4,5.4,19.21
3,4,11,16,0.6,0.6,0.40
4,5,35,16,0.1,0.1,0.04
...,...,...,...,...,...,...
1922,1923,0,56,0.9,0.9,1.38
1923,1924,225,56,0.4,0.4,1.48
1924,1925,276,56,0.1,0.1,1.69
1925,1926,304,56,0.6,0.6,0.32


In [85]:
ene_fosil=pd.merge(carbon,petroleo, on='Id_anio')
ene_fosil

,Id_Energia_Carbon,Id_Pais_x,Id_anio,Consumo_Carbon,Produccion_Carbono,ID_Energia_petroleo,Id_Pais_y,Consumo_Petroleo,Produccion_Petroleo,Reservas_Petroleo
0,1,44,17,0.95,0.97,641,44,4.189146,3.595162,245.938101
1,1,44,17,0.95,0.97,642,160,2.670692,5.700900,323.634963
2,1,44,17,0.95,0.97,643,297,35.857836,22.734377,223.302007
3,1,44,17,0.95,0.97,644,11,1.062918,1.158937,13.459299
4,1,44,17,0.95,0.97,645,35,2.324574,0.491264,9.176795
...,...,...,...,...,...,...,...,...,...,...
57395,1435,210,57,1.19,1.16,2276,124,3.284773,1.545250,0.000000
57396,1435,210,57,1.19,1.16,2277,0,1.703794,1.279520,0.000000
57397,1435,210,57,1.19,1.16,2278,276,2.710886,0.888742,0.000000
57398,1435,210,57,1.19,1.16,2279,304,1.036121,0.428740,0.000000


In [88]:
ene_fosil=pd.merge(ene_fosil,g_n, how='left', on='Id_anio')
ene_fosil

MemoryError: Unable to allocate 18.1 GiB for an array with shape (5, 485833600) and data type int64

In [37]:
#ene_fosil.fillna(0,inplace=True)
#ene_fosil

,Id_Energia_Carbon,Id_Pais_x,Id_anio,Consumo_Carbon,Produccion_Carbono,ID_Energia_petroleo,Id_Pais_y,Consumo_Petroleo,Produccion_Petroleo,Reservas_Petroleo,ID_Gas_Natural,Id_Pais,Reserva_Gas,Produccion_gas,Consumo_gas
0,1,44,17,0.95,0.97,641,44,4.189146,3.595162,245.938101,48.0,44.0,2.5,2.5,1.74
1,1,44,17,0.95,0.97,641,44,4.189146,3.595162,245.938101,49.0,160.0,2.1,2.1,0.87
2,1,44,17,0.95,0.97,641,44,4.189146,3.595162,245.938101,50.0,297.0,5.5,5.5,18.75
3,1,44,17,0.95,0.97,641,44,4.189146,3.595162,245.938101,51.0,11.0,0.6,0.6,0.42
4,1,44,17,0.95,0.97,641,44,4.189146,3.595162,245.938101,52.0,35.0,0.1,0.1,0.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633395,1435,210,57,1.19,1.16,2276,124,3.284773,1.545250,0.000000,0.0,0.0,0.0,0.0,0.00
2633396,1435,210,57,1.19,1.16,2277,0,1.703794,1.279520,0.000000,0.0,0.0,0.0,0.0,0.00
2633397,1435,210,57,1.19,1.16,2278,276,2.710886,0.888742,0.000000,0.0,0.0,0.0,0.0,0.00
2633398,1435,210,57,1.19,1.16,2279,304,1.036121,0.428740,0.000000,0.0,0.0,0.0,0.0,0.00


In [87]:
ene_fosil.drop(columns={'Id_Energia_Carbon','Id_Pais_x','ID_Energia_petroleo','Id_Pais_y','ID_Gas_Natural'},inplace=True)
#ene_fosil_consumo=ene_fosil.loc[:,('Id_anio','Consumo_Carbon','Consumo_Petroleo','Consumo_gas')]
ene_fosil

KeyError: "['ID_Gas_Natural'] not found in axis"

In [27]:
ene_fosil_consumo

,Id_anio,Consumo_Carbon,Consumo_Petroleo,Consumo_gas
0,17,0.95,4.189146,1.74
1,17,0.95,4.189146,0.87
2,17,0.95,4.189146,18.75
3,17,0.95,4.189146,0.42
4,17,0.95,4.189146,0.03
...,...,...,...,...
2633395,57,1.19,3.284773,0.00
2633396,57,1.19,1.703794,0.00
2633397,57,1.19,2.710886,0.00
2633398,57,1.19,1.036121,0.00


In [14]:
ene_fosil_consumo.fillna(0, inplace=True)
ene_fosil_consumo

,Id_anio,Id_Pais,Consumo_Carbon,Consumo_Petroleo,Consumo_gas,Total_Consumo
0,17,44.0,0.95,4.189146,1.74,6.879146
1,17,160.0,0.95,4.189146,0.87,6.009146
2,17,297.0,0.95,4.189146,18.75,23.889146
3,17,11.0,0.95,4.189146,0.42,5.559146
4,17,35.0,0.95,4.189146,0.03,5.169146
...,...,...,...,...,...,...
2633395,57,0.0,1.19,3.284773,0.00,0.000000
2633396,57,0.0,1.19,1.703794,0.00,0.000000
2633397,57,0.0,1.19,2.710886,0.00,0.000000
2633398,57,0.0,1.19,1.036121,0.00,0.000000


In [39]:
ene_fosil['Total_Consumo']=ene_fosil['Consumo_Carbon']+ene_fosil['Consumo_Petroleo']+ene_fosil['Consumo_gas']
ene_fosil

,Id_anio,Consumo_Carbon,Produccion_Carbono,Consumo_Petroleo,Produccion_Petroleo,Reservas_Petroleo,Id_Pais,Reserva_Gas,Produccion_gas,Consumo_gas,Total_Consumo
0,17,0.95,0.97,4.189146,3.595162,245.938101,44.0,2.5,2.5,1.74,6.879146
1,17,0.95,0.97,4.189146,3.595162,245.938101,160.0,2.1,2.1,0.87,6.009146
2,17,0.95,0.97,4.189146,3.595162,245.938101,297.0,5.5,5.5,18.75,23.889146
3,17,0.95,0.97,4.189146,3.595162,245.938101,11.0,0.6,0.6,0.42,5.559146
4,17,0.95,0.97,4.189146,3.595162,245.938101,35.0,0.1,0.1,0.03,5.169146
...,...,...,...,...,...,...,...,...,...,...,...
2633395,57,1.19,1.16,3.284773,1.545250,0.000000,0.0,0.0,0.0,0.00,4.474773
2633396,57,1.19,1.16,1.703794,1.279520,0.000000,0.0,0.0,0.0,0.00,2.893794
2633397,57,1.19,1.16,2.710886,0.888742,0.000000,0.0,0.0,0.0,0.00,3.900886
2633398,57,1.19,1.16,1.036121,0.428740,0.000000,0.0,0.0,0.0,0.00,2.226121


In [40]:
paises = pd.read_sql_query('SELECT *FROM "Paises";', con=conexion)
paises

,Id_Pais,Codigo_pais,Pais
0,1,AFG,Afghanistan
1,2,0,Africa
2,3,0,Africa (BP)
3,4,0,Africa (Shift)
4,5,ALB,Albania
...,...,...,...
309,310,0,World
310,311,YEM,Yemen
311,312,0,Yugoslavia
312,313,ZMB,Zambia


In [44]:
paises[paises['Id_Pais']==57]

,Id_Pais,Codigo_pais,Pais
56,57,COK,Cook Islands


In [41]:

años = pd.read_sql_query('SELECT *FROM "Años";', con=conexion)

In [43]:
ene_fosil_consumo_total=pd.merge(ene_fosil,paises, how='left', on='Id_Pais')
ene_fosil_consumo_total

,Id_anio,Consumo_Carbon,Produccion_Carbono,Consumo_Petroleo,Produccion_Petroleo,Reservas_Petroleo,Id_Pais,Reserva_Gas,Produccion_gas,Consumo_gas,Total_Consumo,Codigo_pais,Pais
0,17,0.95,0.97,4.189146,3.595162,245.938101,44.0,2.5,2.5,1.74,6.879146,CAN,Canada
1,17,0.95,0.97,4.189146,3.595162,245.938101,160.0,2.1,2.1,0.87,6.009146,MEX,Mexico
2,17,0.95,0.97,4.189146,3.595162,245.938101,297.0,5.5,5.5,18.75,23.889146,USA,United States
3,17,0.95,0.97,4.189146,3.595162,245.938101,11.0,0.6,0.6,0.42,5.559146,ARG,Argentina
4,17,0.95,0.97,4.189146,3.595162,245.938101,35.0,0.1,0.1,0.03,5.169146,BRA,Brazil
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2633395,57,1.19,1.16,3.284773,1.545250,0.000000,0.0,0.0,0.0,0.00,4.474773,NaN,NaN
2633396,57,1.19,1.16,1.703794,1.279520,0.000000,0.0,0.0,0.0,0.00,2.893794,NaN,NaN
2633397,57,1.19,1.16,2.710886,0.888742,0.000000,0.0,0.0,0.0,0.00,3.900886,NaN,NaN
2633398,57,1.19,1.16,1.036121,0.428740,0.000000,0.0,0.0,0.0,0.00,2.226121,NaN,NaN


In [21]:

ene_fosil_consumo_total=pd.merge(ene_fosil_consumo_total,años, on='Id_anio')
ene_fosil_consumo_total

,Id_anio,Id_Pais,Consumo_Carbon,Consumo_Petroleo,Consumo_gas,Total_Consumo,Codigo_pais,Pais,Anio
0,17,44.0,0.95,4.189146,1.74,6.879146,CAN,Canada,1981
1,17,44.0,0.95,2.670692,1.74,5.360692,CAN,Canada,1981
2,17,44.0,0.95,35.857836,1.74,38.547836,CAN,Canada,1981
3,17,44.0,0.95,1.062918,1.74,3.752918,CAN,Canada,1981
4,17,44.0,0.95,2.324574,1.74,5.014574,CAN,Canada,1981
...,...,...,...,...,...,...,...,...,...
1903995,56,210.0,1.17,3.121762,0.42,4.711762,0,Other Asia Pacific (BP),2020
1903996,56,210.0,1.17,1.672532,0.42,3.262532,0,Other Asia Pacific (BP),2020
1903997,56,210.0,1.17,2.710886,0.42,4.300886,0,Other Asia Pacific (BP),2020
1903998,56,210.0,1.17,1.100879,0.42,2.690879,0,Other Asia Pacific (BP),2020


In [ ]:
carbon=pd.read_sql_query('SELECT *FROM "Energia_Carbon";', con=conexion)
carbon.rename(columns={'Id_Anio':'Id_anio'},inplace=True)
petroleo=pd.read_sql_query('SELECT *FROM "Energia_petroleo";', con=conexion)
gas=pd.read_sql_query('SELECT *FROM "Gas_Natural";', con=conexion)
ene_fosil=pd.merge(carbon,petroleo, on='Id_anio')
ene_fosil=pd.merge(ene_fosil,gas, on='Id_anio')
ene_fosil.drop(columns={'Id_Pais_x','ID_Energia_petroleo','Id_Pais_y','ID_Gas_Natural'},inplace=True)
ene_fosil_consumo=ene_fosil.loc[:,('Id_anio','Id_Pais','Consumo_Carbon','Consumo_Petroleo','Consumo_gas')]
ene_fosil_consumo['Total_Consumo']=ene_fosil_consumo['Consumo_Carbon']+ene_fosil_consumo['Consumo_Petroleo']+ene_fosil_consumo['Consumo_gas']
ene_fosil_consumo_total=pd.merge(ene_fosil_consumo,paises, on='Id_Pais')
ene_fosil_consumo_total=pd.merge(ene_fosil_consumo_total,años, on='Id_anio')
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Europe (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other CIS (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Middle East (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Western Africa (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Northern Africa (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
indexN=ene_fosil_consumo_total[ene_fosil_consumo_total['Pais']=='Other Asia Pacific (BP)'].index
ene_fosil_consumo_total.drop(indexN, inplace=True)
año90=ene_fosil_consumo_total[ene_fosil_consumo_total.Anio>=1990]
pais=año90[(año90.Pais==pais)].groupby(by=['Anio']).mean()

fig = go.Figure()
fig.add_trace(go.Bar(
x=pais.index,
y=pais['Consumo_Carbon'],
name='Carbon',
marker_color= 'lightslategray'
    ))
fig.add_trace(go.Bar(
x=pais.index,
y=pais['Consumo_Petroleo'],
name='Petroleo',
marker_color='black'
    ))
fig.add_trace(go.Bar(
x=pais.index,
y=pais['Consumo_gas'],
name='Gas Natural',
marker_color='yellowgreen'
    ))
return fig.show()